# Backtesting and Evaluation

Create rolling splits and evaluate a simple baseline without leakage.


In [ ]:
from pathlib import Path
import sys

root = Path.cwd()
if (root / 'src').exists():
    sys.path.insert(0, str(root))
elif (root.parent / 'src').exists():
    sys.path.insert(0, str(root.parent))
import pandas as pd
import numpy as np

from src.chapter2.backtesting import RollingWindowBacktest
from src.chapter2.evaluation import ForecastMetrics

ds = pd.date_range('2024-01-01', periods=240, freq='H')
y = 10 + 0.1 * np.arange(len(ds)) + np.random.normal(scale=0.5, size=len(ds))
df = pd.DataFrame({'unique_id': 'series_1', 'ds': ds, 'y': y})

backtest = RollingWindowBacktest(min_train_size=120, test_size=24, step_size=24)
splits = backtest.generate_splits(df, unique_id='series_1')
print(splits[0].info)

# Simple baseline: repeat last training value for the horizon
split = splits[0]
train = df.iloc[split.train_indices]
test = df.iloc[split.test_indices]
yhat = np.repeat(train['y'].iloc[-1], len(test))
rmse = ForecastMetrics.rmse(test['y'].values, yhat)
print('RMSE:', rmse)
